In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
# matplotlib 한글 꺠짐 방지
# apt-get update
# apt-get install fonts-nanum* 
# apt-get install fontconfig
# fc-cache -fv  # font 캐시 날리기
# rm -rf /home/hy1/.cache/matplotlib/  #matplotliob 폰트 캐시 날리기



In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 


In [5]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [6]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='NanumGothic')

## Load - DataWareHouse

## JDBC DICT

In [7]:
JDBC = {
    'url':'jdbc:oracle:thin:@decorona1_high?TNS_ADMIN=/home/big/project/db/Wallet_DECORONA1'
    ,'props':{
        'user':'dw_jjy'
       ,'password':'123qwe!@#QWE'
    }   
}

### 1. LOC 테이블 저장

In [8]:
area = spark.read.csv('/corona_data/loc/sido_area.csv', encoding='CP949', header=True)
popu = spark.read.csv('/corona_data/loc/sido_population.csv', encoding='CP949', header=True)



AnalysisException: Path does not exist: hdfs://localhost:9000/corona_data/loc/sido_population.csv

In [ ]:
# loc 데이터 저장
area_pop = area.join(popu, on='loc')
area_pop = area_pop.select(col('loc').alias('LOC')
                                   , col('area').alias('AREA')
                                   ,col('total').alias('POPULATION')
                          )
area_pop.show()
area_pop.write.jdbc(url=JDBC['url'], table='LOC', mode='append', properties=JDBC['props'])

### 2. CORONA_PATIENT 테이블 저장

In [9]:
#코로나 감염자 데이터 전체 파일조회
path = '/corona_data/patient'
co_patient_json = spark.read.json(path, encoding='UTF-8')
co_patient_json.show()

+----------------------+---------+------+----------+--------------+----------+
|                 items|numOfRows|pageNo|resultCode|     resultMsg|totalCount|
+----------------------+---------+------+----------+--------------+----------+
|[{16, 12659, 검역, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|  [{1465, 1461477, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|[{16, 12440, 검역, ...|      500|     1|        00|NORMAL SERVICE|     19684|
| [{1043, 868166, 충...|      500|     1|        00|NORMAL SERVICE|     19684|
|  [{1429, 1214957, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|[{528, 687990, 전남...|      500|     1|        00|NORMAL SERVICE|     19684|
|  [{25441, 20983169...|      500|     1|        00|NORMAL SERVICE|     19684|
| [{1376, 847452, 대...|      500|     1|        00|NORMAL SERVICE|     19684|
|[{520, 666170, 전남...|      500|     1|        00|NORMAL SERVICE|     19684|
|  [{5077, 4264423, ...|      500|     1|        00|NORMAL SER

In [10]:
data = []
for r1 in co_patient_json.select('items').toLocalIterator():
    if not r1.items:
        continue
    for r2 in r1.items:
        data.append(r2)

patient_data = spark.createDataFrame(data)
patient_data.show(3)

co_patients = patient_data.select(
    patient_data.gubun.alias('LOC')
    ,patient_data.deathCnt.alias('DEATH_CNT')
     ,patient_data.defCnt.alias('DEF_CNT')
     ,patient_data.localOccCnt.alias('LOC_OCC_CNT')
     ,patient_data.qurRate.alias('QUR_RATE')
     ,patient_data.stdDay.alias('STD_DAY')
).where(~(col('LOC').isin(['합계','검역'])) ).distinct()
co_patients.printSchema()


+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
|deathCnt| defCnt|gubun|gubunCn|    gubunEn|incDec|isolClearCnt|isolIngCnt|localOccCnt|overFlowCnt|qurRate|    stdDay|
+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
|      16|  12659| 검역| 隔離區|  Lazaretto|    17|           0|         0|          0|         17|      -|2022-08-25|
|      16|  12659| 검역| 隔離區|  Lazaretto|    17|           0|         0|          0|         17|      -|2022-08-25|
|    6715|6109867| 경기|   京畿|Gyeonggi-do| 27032|           0|         0|      27007|         25|  45040|2022-08-25|
+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
only showing top 3 rows

root
 |-- LOC: string (nullable = true)
 |-- DEATH_CNT: string (nullable = true)
 |-- DEF_CNT: string (nullable = true)
 |-- LOC_OCC_CNT: string (nullable =

In [11]:
co_patients.write.jdbc(url=JDBC['url'], table='CORONA_PATIENTS', mode='append', properties=JDBC['props'])

### 3.  CORONA_VACCINE 테이블 저장

In [12]:
file_name = '/corona_data/vaccine/'
vaccine= spark.read.json(file_name, multiLine=True)
vaccine.select(vaccine.data).show()


# vaccine.show()


+----------------------+
|                  data|
+----------------------+
|[{서울, 8340437, 82...|
+----------------------+



In [13]:
vaccine.select('meta').collect()

[Row(meta=Row(cols=Row(loc='지역', v1='1차접종자수', v2='2차접종자수', v3='3차접종자수', v4='4차접종자수'), desc='지역별 코로나 예방접종 인구 현황', std_day='2022-09-18'))]

In [14]:
data = []
for r1 in vaccine.select(vaccine.data, vaccine.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        # row 객체를 만들기 위해 dict 생성
        # ** 파이썬 압축해제 키워드 
        # **kwargs   => 여러 쌍의 키워드 args 가 넘어오면 받아서 dictionary로 반환
        # fnc(**dict) => dict에 있는 key-value 값들이 여러쌍의 kwargs  형태로 함수에 전달
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day']
        data.append(Row(**temp))      
        
        
vaccine_data = spark.createDataFrame(data)
vaccine_data.show(3)


+----+-------+-------+-------+-------+----------+
| loc|     v1|     v2|     v3|     v4|   std_day|
+----+-------+-------+-------+-------+----------+
|서울|8340437|8261615|6058568|1231362|2022-09-18|
|부산|2879935|2851047|2134874| 490187|2022-09-18|
|대구|2019298|1996365|1402789| 260381|2022-09-18|
+----+-------+-------+-------+-------+----------+
only showing top 3 rows



In [15]:
#column을 row로 :stack
#row를 column : pivot
pd_vaccine = vaccine_data.to_pandas_on_spark() #판다스로 변환
pd_vaccine=pd_vaccine.set_index(['loc','std_day'])
pd_vaccine=pd_vaccine.stack()

#시리즈나 데이터프레임으로 반환되는데, 이때 아래 사용
pd_vaccine = pd_vaccine.to_dataframe('V_CNT') #0이 컬럼명에 들어가있는거 확인 ->V_CNT로 바꾸기
#
#3Spd_vaccine #판다스일때 show 사용하지 않는다
pd_vaccine

V_CNT
loc std_day                
서울  2022-09-18 v2   8261615
               v4   1231362
               v1   8340437
               v3   6058568
부산  2022-09-18 v2   2851047
               v4    490187
               v1   2879935
               v3   2134874
대구  2022-09-18 v2   1996365
               v4    260381
               v1   2019298
               v3   1402789
인천  2022-09-18 v2   2546449
               v4    396104
               v1   2570971
               v3   1908028
광주  2022-09-18 v2   1251041
               v4    234405
               v1   1262686
               v3    968138
대전  2022-09-18 v2   1233926
               v4    193829
               v1   1246195
               v3    907679
울산  2022-09-18 v2    959274
               v4    125398
               v1    968616
               v3    719551
세종  2022-09-18 v2    294854
               v4     34695
               v1    298612
               v3    209194
경기  2022-09-18 v2  11712435
               v4   1688068
               v1  11828148
               v3   8678460
강원  2022-09-18 v2   1338994
               v4    270055
               v1   1350365
               v3   1062198
충북  2022-09-18 v2   1417015
               v4    278087
               v1   1429660
               v3   1106015
충남  2022-09-18 v2   1881852
               v4    366884
               v1   1899521
               v3   1475649
전북  2022-09-18 v2   1580606
               v4    371028
               v1   1593762
               v3   1284213
전남  2022-09-18 v2   1640127
               v4    445122
               v1   1654761
               v3   1363331
경북  2022-09-18 v2   2269467
               v4    395869
               v1   2293393
               v3   1724688
경남  2022-09-18 v2   2854198
               v4    485197
               v1   2883579
               v3   2142332
제주  2022-09-18 v2    584841
               v4     92385
               v1    590587
               v3    437858

In [16]:
pd_vaccine = pd_vaccine.reset_index()
pd_vaccine=pd_vaccine.rename(columns={'level_2':'V_TH'})
vaccine_data=pd_vaccine.to_spark()

vaccine_data.drop('level_0')
vaccine_data.drop('index')
vaccine_data.show(3)

DataFrame[loc: string, std_day: string, V_TH: string, V_CNT: string]

DataFrame[loc: string, std_day: string, V_TH: string, V_CNT: string]

+----+----------+----+-------+
| loc|   std_day|V_TH|  V_CNT|
+----+----------+----+-------+
|서울|2022-09-18|  v2|8261615|
|서울|2022-09-18|  v4|1231362|
|서울|2022-09-18|  v1|8340437|
+----+----------+----+-------+
only showing top 3 rows



In [17]:
tmp=pd_vaccine.to_spark()
tmp.show()
#쉼표 안사려져서 저장할떄 오류 생김

+----+----------+----+-------+
| loc|   std_day|V_TH|  V_CNT|
+----+----------+----+-------+
|서울|2022-09-18|  v2|8261615|
|서울|2022-09-18|  v4|1231362|
|서울|2022-09-18|  v1|8340437|
|서울|2022-09-18|  v3|6058568|
|부산|2022-09-18|  v2|2851047|
|부산|2022-09-18|  v4| 490187|
|부산|2022-09-18|  v1|2879935|
|부산|2022-09-18|  v3|2134874|
|대구|2022-09-18|  v2|1996365|
|대구|2022-09-18|  v4| 260381|
|대구|2022-09-18|  v1|2019298|
|대구|2022-09-18|  v3|1402789|
|인천|2022-09-18|  v2|2546449|
|인천|2022-09-18|  v4| 396104|
|인천|2022-09-18|  v1|2570971|
|인천|2022-09-18|  v3|1908028|
|광주|2022-09-18|  v2|1251041|
|광주|2022-09-18|  v4| 234405|
|광주|2022-09-18|  v1|1262686|
|광주|2022-09-18|  v3| 968138|
+----+----------+----+-------+
only showing top 20 rows



In [18]:
tmp.write.jdbc(url=JDBC['url'], table='CORONA_VACCINE', mode='append', properties=JDBC['props'])

### 4. LOC_FACILITY_CNT	테이블 저장